In [1]:
from haystack.utils import  convert_files_to_dicts, print_answers
from haystack.nodes import FARMReader
from haystack.document_stores import FAISSDocumentStore
from haystack.nodes import DensePassageRetriever
from haystack.pipelines import ExtractiveQAPipeline
from pymongo import MongoClient
from haystack.schema import Document


C:\Users\ANT-PC\anaconda3\envs\ds\lib\site-packages\torchaudio\backend\utils.py:67: UserWarning: No audio backend is available.
  warnings.warn('No audio backend is available.')
C:\Users\ANT-PC\anaconda3\envs\ds\lib\site-packages\ray\autoscaler\_private\cli_logger.py:57: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  warnings.warn(


In [2]:

db_client = MongoClient(host="localhost", port=27017)
database = db_client['Website_Chatbot']
collection = database["MitsSpider"]


In [9]:
def split_paragraph(text):
    print(text)
    # for para in text.split("\n\n"):
    #     if not para.strip():
    #         continue
        # print(para)

In [11]:
docs = [] 
for document in collection.find({}):
    content = document["content"]
    for para in content.split("\n\n"):
        if not para.strip():
            continue

        tmp_doc = Document(content=para)
        tmp_doc.id = str(document["_id"])
        tmp_doc.meta = {"source":document['source'], }
        tmp_doc.content_type = "str"
        docs.append(tmp_doc)


In [12]:
len(docs)

98

In [13]:
document_store = FAISSDocumentStore(faiss_index_factory_str="Flat")
document_store.write_documents(docs)

Writing Documents: 10000it [00:00, 37854.70it/s]         


In [14]:
retriever = DensePassageRetriever(document_store=document_store,
                                 query_embedding_model='facebook/dpr-question_encoder-single-nq-base',
                                 passage_embedding_model='facebook/dpr-ctx_encoder-single-nq-base',
                                 max_seq_len_query=64,
                                 max_seq_len_passage=256,
                                 batch_size=16,
                                 use_gpu=True,
                                 embed_title=True,
                                 use_fast_tokenizers=True)
document_store.update_embeddings(retriever)

INFO - haystack.modeling.utils -  Using devices: CUDA:0
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Could not find facebook/dpr-question_encoder-single-nq-base locally.
INFO - haystack.modeling.model.language_model -  Looking on Transformers Model Hub (in local cache and online)...
INFO - haystack.modeling.model.language_model -  Loaded facebook/dpr-question_encoder-single-nq-base
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Could not find facebook/dpr-ctx_encoder-single-nq-base locally.
INFO - haystack.modeling.model.language_model -  Looking on Transformers Model Hub (in local cache and online)...
INFO - haystack.modeling.model.language_model -  Loaded facebook/dpr-ctx_encoder

In [15]:
reader = FARMReader(model_name_or_path="Saved Models/roberta_base_squad2", use_gpu=True,
num_processes=0)

INFO - haystack.modeling.utils -  Using devices: CUDA
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Model found locally at Saved Models/roberta_base_squad2
INFO - haystack.modeling.model.language_model -  Loaded Saved Models/roberta_base_squad2
INFO - haystack.modeling.model.adaptive_model -  Found files for loading 1 prediction heads
WARNING - haystack.modeling.model.prediction_head -  Some unused parameters are passed to the QuestionAnsweringHead. Might not be a problem. Params: {"training": false, "num_labels": 2, "ph_output_type": "per_token_squad", "model_type": "span_classification", "label_tensor_name": "question_answering_label_ids", "label_list": ["start_token", "end_token"], "metric": "squad", "name": "QuestionAnsweringHead"}
INFO - haystack.modeling.model.prediction_head -  Loading prediction head

In [16]:
pipeline = ExtractiveQAPipeline(reader=reader,retriever=retriever)

In [18]:
prediction = pipeline.run(query="Where is mits located?",
                         params = {"Retriever":{"top_k":10}, 
                                  "Reader":{"top_k":10}})

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]C:\Users\ANT-PC\anaconda3\envs\ds\lib\site-packages\haystack\modeling\model\prediction_head.py:462: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  start_indices = flat_sorted_indices // max_seq_len
Inferencing Samples: 100%|██████████| 1/1 [00:01<00:00,  1.07s/ Batches]


In [25]:
first_ans = prediction["answers"][0]
first_ans.document_id

'6213cd05abf8a36f04f5c720'

### Saving the models

In [26]:
retriever.save("context_model_retriever_2")
document_store.save("document_store")

INFO - haystack.modeling.model.biadaptive_model -  prediction_head saving


In [2]:
tmp_doc_store = FAISSDocumentStore.load("document_store")
tmp_reader = FARMReader(model_name_or_path="Saved Models/roberta_base_squad2",
use_gpu=True, num_processes=0)
tmp_retriever = DensePassageRetriever.load("context_model_retriever_2", tmp_doc_store)
tmp_pipeline = ExtractiveQAPipeline(tmp_reader, tmp_retriever)

INFO - haystack.modeling.utils -  Using devices: CUDA
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Model found locally at Saved Models/roberta_base_squad2
INFO - haystack.modeling.model.language_model -  Loaded Saved Models/roberta_base_squad2
INFO - haystack.modeling.model.adaptive_model -  Found files for loading 1 prediction heads
WARNING - haystack.modeling.model.prediction_head -  Some unused parameters are passed to the QuestionAnsweringHead. Might not be a problem. Params: {"training": false, "num_labels": 2, "ph_output_type": "per_token_squad", "model_type": "span_classification", "label_tensor_name": "question_answering_label_ids", "label_list": ["start_token", "end_token"], "metric": "squad", "name": "QuestionAnsweringHead"}
INFO - haystack.modeling.model.prediction_head -  Loading prediction head

In [3]:
predictions = tmp_pipeline.run("Where is MITS located")

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]C:\Users\ANT-PC\anaconda3\envs\ds\lib\site-packages\haystack\modeling\model\prediction_head.py:462: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  start_indices = flat_sorted_indices // max_seq_len
Inferencing Samples: 100%|██████████| 1/1 [00:01<00:00,  1.26s/ Batches]


In [7]:
answers = predictions["answers"]


In [9]:
answers[0].document_id

'6213cd05abf8a36f04f5c720'